## Chapter 4 – Exercise 4:  Sử dụng Mysql Connector để thực hiện các truy vấn có tham số

#### Học viên thực hiện các yêu cầu sau và đối chiếu với kết quả cho trước (nếu có):

In [4]:
from mysql.connector import connect

In [6]:
conn = connect(host='localhost', port=3306, database='salesdb', user='root', password='')

In [10]:
# Câu 1: Liệt kê danh sách các sản phẩm do công ty 'Tokyo Traders' cung cấp, với tham số tên công ty do người dùng nhập vào

supplier = input('Input a company name: ')

data_1 = {'company_name':supplier}

sql_cmd1 = '''
SELECT p.Id as ProductId, p.ProductName, p.SupplierId, p.UnitPrice, 
	   p.Package, p.IsDiscontinued, s.CompanyName
FROM `suppliers` s JOIN `products` p ON s.Id=p.SupplierId
WHERE s.CompanyName = %(company_name)s;
'''


curs1 = conn.cursor()

curs1.execute(sql_cmd1, data_1)

row = curs1.fetchone()
while row:
    print(row)
    row = curs1.fetchone()

Input a company name:  Tokyo Traders


(9, 'Mishi Kobe Niku', 4, Decimal('97.00'), '18 - 500 g pkgs.', 1, 'Tokyo Traders')
(10, 'Ikura', 4, Decimal('31.50'), '12 - 200 ml jars', 0, 'Tokyo Traders')
(74, 'Longlife Tofu', 4, Decimal('10.00'), '5 kg pkg.', 0, 'Tokyo Traders')
(79, 'Shushi', 4, Decimal('4.00'), '1 box x 2 bags', 1, 'Tokyo Traders')


<details>
  <summary>Nhấn vào đây để xem kết quả!</summary>
  <pre>Nhập tên nhà cung cấp: Tokyo Traders
(9, 'Mishi Kobe Niku', 4, Decimal('97.00'), '18 - 500 g pkgs.', 1, 'Tokyo Traders')
(10, 'Ikura', 4, Decimal('31.00'), '12 - 200 ml jars', 0, 'Tokyo Traders')
(74, 'Longlife Tofu', 4, Decimal('10.00'), '5 kg pkg.', 0, 'Tokyo Traders')
</pre>
  
</details>

In [17]:
# Câu 2: Cập nhật đơn giá tăng thêm 3usd cho các sản phẩm do công ty 'Tokyo Traders' cung cấp, 
# với tham số đơn giá tăng và tên công ty do người dùng nhập vào

supplier = input('Input a company name: ')
add_price = input('Input the amount of added price: ')

data_2 = {'company_name':supplier, 'increase_price':add_price}

sql_cmd2 = '''
SELECT p.Id as ProductId, p.ProductName, p.SupplierId, (p.UnitPrice + %(increase_price)s) as New_UnitPrice, 
	   p.Package, p.IsDiscontinued, s.CompanyName
FROM `suppliers` s JOIN `products` p ON s.Id=p.SupplierId
WHERE s.CompanyName = %(company_name)s;
'''


curs2 = conn.cursor()

curs2.execute(sql_cmd2, data_2)

row = curs2.fetchone()
while row:
    print(row)
    row = curs2.fetchone()

Input a company name:  Tokyo Traders
Input the amount of added price:  3


(9, 'Mishi Kobe Niku', 4, 100.0, '18 - 500 g pkgs.', 1, 'Tokyo Traders')
(10, 'Ikura', 4, 34.5, '12 - 200 ml jars', 0, 'Tokyo Traders')
(74, 'Longlife Tofu', 4, 13.0, '5 kg pkg.', 0, 'Tokyo Traders')


<details>
  <summary>Nhấn vào đây để xem kết quả!</summary>
  <pre>Nhập giá trị đơn giá tăng thêm: 3
Nhập tên nhà cung cấp: Tokyo Traders
</pre>
  
</details>

In [34]:
# Câu 3: Thêm sản phẩm mới vào bảng Products với các giá trị tương ứng do người dùng nhập vào. 
# Sau khi thêm, xuất thông báo hỏi người dùng có thêm tiếp hay không (Y/N). 
# Nếu trả lời Y sẽ cho phép thêm tiếp sản phẩm mới, nếu trả lời N sẽ thoát quá trình thêm sản phẩm.

data_list = []
while True:
    print()
    prod_id = input('Input a new product id: ')
    prod_name = input('Input a new product name: ')
    supp_id = input('Input a supplier id: ')
    uprice = input('Input a unit price: ')
    pkg = input('Input a new packaging: ')
    dis_ctn = input('Input discontinue status 0/1: ')

    value_dict = {
        'product_id':prod_id,
        'product_name':prod_name,
        'supplier_id':supp_id,
        'unit_price':uprice,
        'package':pkg,
        'discontinue':dis_ctn
    }
    data_list.append(value_dict)
    
    tt = input('Do you want to continue? Y/N: ')
    if tt == 'Y': continue
    else: break

curs3 = conn.cursor()

sql_cmd3='''
INSERT INTO `products` (`Id`, `ProductName`, `SupplierId`, `UnitPrice`, `Package`, `IsDiscontinued`)
VALUES(%(product_id)s, %(product_name)s, %(supplier_id)s, %(unit_price)s, %(package)s, %(discontinue)s);
'''

curs3.executemany(sql_cmd3, data_list)
curs3.close()

for i in range(len(data_list)):
    prdname = data_list[i].get('product_name')
    curs3 = conn.cursor()
    curs3.execute(f"SELECT * FROM `products` WHERE ProductName='{prdname}'")
    row = curs3.fetchone()
    print(row)
    curs3.close()

Input a new product id:  80
Input a new product name:  Salmon
Input a supplier id:  6
Input a unit price:  20
Input a new packaging:  1 box x 1 kg
Input discontinue status 0/1:  0
Do you want to continue? Y/N:  Y
Input a new product id:  81
Input a new product name:  Marsmellow
Input a supplier id:  8
Input a unit price:  5
Input a new packaging:  1 bag x 20 pieces
Input discontinue status 0/1:  0
Do you want to continue? Y/N:  N


(80, 'Salmon', 6, Decimal('20.00'), '1 box x 1 kg', 1)
(81, 'Marsmellow', 8, Decimal('5.00'), '1 bag x 20 pieces', 1)


In [12]:
# Câu 4: Xóa thông tin đơn đặt hàng theo số đặt hàng (OrderId), trong đó số đặt hàng do người dùng nhập vào

ord_id_delete = input('Input an order id to delete: ')

sql_cmd4 = f'''
DELETE FROM `orders` WHERE Id = {int(ord_id_delete)};
DELETE FROM `orderitems` WHERE OrderId = {int(ord_id_delete)};
'''

curs4 = conn.cursor()
curs4.execute(sql_cmd4)
curs4.close()

Input an order id to delete:  1


True

In [14]:
conn.close()